In [5]:
import multiprocessing
import xgboost as xgb
import pandas as pd
import numpy as np
import pickle 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import Imputer
import warnings
warnings.filterwarnings("ignore")
#用来计算程序运行时间
import datetime
starttime = datetime.datetime.now()

In [2]:
#读取Y
all_df_y = pickle.load(open("./dump_file/all_df_y","rb"))
all_df_y['2cArousal'] = 0
all_df_y['2cArousal'][all_df_y['valence'] >= 5] = 1
all_df_y['2cValence'] = 0
all_df_y['2cValence'][all_df_y['valence'] >= 5] = 1
print(all_df_y.head(5))
y = all_df_y[["2cValence"]]

print("######读取特征（基于EEG）######")
#EEG特征表
eegFeatureDF = pickle.load(open("./dump_file/eegFeatureDF","rb"))

if True:
    #加上早期的统计特征
    for eegCH in range(1,33,1):
        file_path = "./dump_file/CH{}_eeg_feat_df".format(eegCH)
        tmpDF = pickle.load(open(file_path,"rb"))
        eegFeatureDF = pd.concat([eegFeatureDF,tmpDF],axis=1)
if True:#加上样本熵特征
    filePath = "./dump_file_sampEn/sampEnFeatures".format(eegCH)
    tmpDF = pickle.load(open(filePath,"rb"))
    eegFeatureDF = pd.concat([eegFeatureDF,tmpDF],axis=1)

       valence  arousal  dominance  liking  2cArousal  2cValence
s01_0     7.71     7.60       6.90    7.83          1          1
s01_1     8.10     7.31       7.28    8.47          1          1
s01_2     8.58     7.54       9.00    7.08          1          1
s01_3     4.94     6.01       6.12    8.06          0          0
s01_4     6.96     3.92       7.19    6.05          1          1
######读取特征（基于EEG）######


In [7]:
for seed in [0,100,200,300,400,500,600,700,800,900]:
    print("[seed:{}]****************************************************".format(seed))
    xTrainIdx = pickle.load(open("./dump_file/xTrainIdx_{}".format(seed),"rb"))
    xTestIdx = pickle.load(open("./dump_file/xTestIdx_{}".format(seed),"rb"))
    trainY = y.loc[xTrainIdx]
    testY = y.loc[xTestIdx]
    trainX = eegFeatureDF.loc[xTrainIdx]
    testX = eegFeatureDF.loc[xTestIdx]    
    data = trainX
    target = trainY   
    print("######xgb classification ######")
    xgb_model = xgb.XGBClassifier(max_depth=5,learning_rate=0.1,n_estimators=50,
                                      objective='binary:logistic',booster='gbtree',n_jobs=10,
                                      subsample=0.9, colsample_bytree=0.9, colsample_bylevel=0.9,
                                      reg_alpha=0.5, reg_lambda=1.0,gamma=0,
                                      scale_pos_weight=1)
    xgb_model.fit(trainX.values,trainY.values)
    predY = xgb_model.predict(testX.values)
    print("ACC",accuracy_score(y_true=testY, y_pred=predY))
    print("F1",f1_score(y_true=testY, y_pred=predY))
    print("Recal",recall_score(y_true=testY,y_pred=predY))
    print("Precision",precision_score(y_true=testY, y_pred=predY))

[seed:0]****************************************************
######xgb classification ######
ACC 0.6536458333333334
F1 0.712742980561555
Recal 0.7603686635944701
Precision 0.6707317073170732
[seed:100]****************************************************
######xgb classification ######
ACC 0.6302083333333334
F1 0.7029288702928871
Recal 0.7741935483870968
Precision 0.6436781609195402
[seed:200]****************************************************
######xgb classification ######
ACC 0.6119791666666666
F1 0.6823027718550106
Recal 0.7373271889400922
Precision 0.6349206349206349
[seed:300]****************************************************
######xgb classification ######
ACC 0.6692708333333334
F1 0.7315010570824525
Recal 0.7972350230414746
Precision 0.67578125
[seed:400]****************************************************
######xgb classification ######
ACC 0.6223958333333334
F1 0.7034764826175869
Recal 0.7926267281105991
Precision 0.6323529411764706
[seed:500]*******************************